<a href="https://colab.research.google.com/github/data04190/News_Classification/blob/main/KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 104 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=5a54c5e6ccc53f60ff9d3729e05a3ea5745411836deee75822b0a941405bc82b
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 34.7 MB/s 
     |████████████████████████████████| 3.0 MB 36.8 MB/s 


In [ ]:
# KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xuft5d3q
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xuft5d3q
     |████████████████████████████████| 131 kB 4.3 MB/s 
     |████████████████████████████████| 4.5 MB 33.1 MB/s 
     |████████████████████████████████| 3.5 MB 28.9 MB/s 
     |████████████████████████████████| 6.8 MB 28.0 MB/s 
     |████████████████████████████████| 596 kB 34.7 MB/s 
     |████████████████████████████████| 67 kB 2.5 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 8.5 MB 26.5 MB/s 
     |████████████████████████████████| 127 kB 32.7 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=676bc329faffbfad0727e29c334d40af6ac0726fb75460efdf4773afa55fadd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-4jc2ukn7/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [ ]:
#라이브러리 불러오기
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from datetime import datetime, timezone, timedelta
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm

# koBert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 시드 고정
RANDOM_SEED = 2022
torch.manual_seed(RANDOM_SEED)    #random seed 고정
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 경로 설정
DATA_DIR = '/content/drive/MyDrive/AI_hub/data/' # 코드와 데이터가 위치한 경로 지정 #####

train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

In [ ]:
train.head()

,text,category
0,이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...,연예/문화
1,"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...",연예/문화
2,인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...,생활
3,김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...,정치
4,"""지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...",스포츠


In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#카테고리형 데이터(Categorical Data)를 수치형 데이터(Numerical Data)로 변환

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()   
encoder.fit(train['category'])
train['category'] = encoder.transform(train['category'])   
train.head()

,text,category
0,이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...,4
1,"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...",4
2,인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...,2
3,김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...,5
4,"""지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...",3


In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '경제', 1: '사회', 2: '생활', 3: '스포츠', 4: '연예/문화', 5: '정치'}

In [ ]:
data_list = []
for q, label in zip(train['text'], train['category'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# Setting parameters
max_len = 128 # 해당 길이를 초과하는 단어에 대해서는 bert가 학습하지 않음
batch_size = 32 

In [ ]:
class CustomDataset(Dataset):
   
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#train & val 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_val = train_test_split(data_list, test_size=0.1, random_state=42, )

In [ ]:
print(len(dataset_train))
print(len(dataset_val))

33883
3765


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = CustomDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = CustomDataset(dataset_val, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=5)
valid_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, shuffle=True, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Hyper-parameters
learning_rate =  2e-5
num_epochs = 10
log_interval = 200
warmup_ratio = 0.1

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   #6개분류
                 dr_rate=None,
                 params=None):
        super(BertClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BertClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
criterion = 0 #initialization
print('Training Start...')

for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train loss {} train acc {}".format(e+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        val_acc += calc_accuracy(out, label)
        
    print("epoch {} valid loss {} valid acc {}".format(e+1, loss.data.cpu().numpy(), val_acc / (batch_id+1)))
    if val_acc / (batch_id+1) > criterion:
        # 모델이 개선됨 -> 검증 점수와 weight 갱신
        criterion = val_acc / (batch_id+1)
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        # 모델이 개선됨 -> 모델 저장
        print("Update model save...")
        torch.save(check_point, os.path.join(DATA_DIR, 'model_best.pt'))
        
print('Training Finish!')

Training Start...


  0%|          | 0/1059 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.793130874633789 train acc 0.1875
epoch 1 batch id 201 loss 1.6970341205596924 train acc 0.25404228855721395
epoch 1 batch id 401 loss 0.8828188180923462 train acc 0.38154613466334164
epoch 1 batch id 601 loss 0.7661982774734497 train acc 0.4923044925124792
epoch 1 batch id 801 loss 0.9738209247589111 train acc 0.5562968164794008
epoch 1 batch id 1001 loss 0.8094286918640137 train acc 0.5976211288711288
epoch 1 train loss 0.47806403040885925 train acc 0.6077208582520197


  0%|          | 0/118 [00:00<?, ?it/s]

...

epoch 9 valid loss 0.6765162348747253 valid acc 0.8303571428571429


  0%|          | 0/1059 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.017399996519088745 train acc 1.0
epoch 10 batch id 201 loss 0.23214025795459747 train acc 0.9853855721393034
epoch 10 batch id 401 loss 0.20108339190483093 train acc 0.9867518703241895
epoch 10 batch id 601 loss 0.004581042565405369 train acc 0.9868448419301165
epoch 10 batch id 801 loss 0.17270325124263763 train acc 0.9878277153558053
epoch 10 batch id 1001 loss 0.05262422561645508 train acc 0.9881056443556444
epoch 10 train loss 0.2706531882286072 train acc 0.9880729112020425


  0%|          | 0/118 [00:00<?, ?it/s]

epoch 10 valid loss 1.299759864807129 valid acc 0.8280871670702179
Training Finish!


In [ ]:
# 테스트 데이터셋 클래스 정의
class TestDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.categories = mapping
        self.num_labels = len(self.categories)
        
        self.sentences = [transform([i]) for i in dataset]
        
    def label_decoder(self, labels):
        try:
            labels = list(map(lambda x : self.categories[x], labels))
            return labels
        except:
            assert 'Invalid intent'
            
    def __len__(self):
        return (len(self.sentences))
    
    def __getitem__(self, i):
        return (self.sentences[i])

In [ ]:
# 테스트 데이터 로드 및 출력 확인
test_data = test.copy()
test_data.head(10)

,ID,text
0,0,"""[기자] 지난 24일 국회 국방위원회에서 현역 대령인 기무부대장이 송영무 국방장관..."
1,1,[IMG1] 한예슬-에릭 커플의 드라마 '스파이명월'의 시청률이 3회연속 하락세를 ...
2,2,"""디지털 광장입니다. 운전대를 잡는 시간보다 구르고 넘어지는 시간이 더 많은 극한의..."
3,3,정부가 최저 임금 인상 대책으로 자영업자와 영세사업자에게 지원하는 일자리 안정자금의...
4,4,청와대가 1박2일간 숨가빴던 방북 일정을 마치고 돌아온 대북특별사절단의 후일담을 8...
5,5,서울시는 올해 2000억원을 포함해 2022년까지 민관이 참여하는 혁신성장펀드 1조...
6,6,"""성폭행 의혹을 받는 안희정 전 충남지사가 오늘(8일) 오후에 예정됐던 기자회견을 ..."
7,7,"소지섭♡손예진 ‘지만갑’의 아역 김지환, 또 다른 숨은 주인공영화 ‘지금 만나러 갑..."
8,8,이이경(30)과의 열애를 인정한 배우 정인선(28)이 화제인 가운데 과거 이력도 주...
9,9,“불투명한 회계 문제가 원인 국책硏 이제 경각심 가져야” 청와대도 압력설 전면 부인...


In [ ]:
# 테스트 데이터 로드
dataset_test = test_data['text'].tolist()
data_test = TestDataset(dataset_test, 0, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
MODEL_DIR = os.path.join(DATA_DIR, 'model_best.pt')
model = BertClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load(MODEL_DIR)['model'])

model.eval()
pred = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length.long().to(device)

    out = model(token_ids, valid_length, segment_ids)
    pred.extend(out.argmax(dim=1).tolist())
    
    # 진행과정 출력
    if batch_id % 10 == 0:
        print(f'Prediction: {batch_id}/{len(test_dataloader)} completed')

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred = data_test.label_decoder(pred)
print('decode Completed!')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Prediction: 0/97 completed
Prediction: 10/97 completed
Prediction: 20/97 completed
Prediction: 30/97 completed
Prediction: 40/97 completed
Prediction: 50/97 completed
Prediction: 60/97 completed
Prediction: 70/97 completed
Prediction: 80/97 completed
Prediction: 90/97 completed
decode Completed!


In [ ]:
# 제출 파일 확인
submit['category'] = pred
submit

,ID,category
0,0,정치
1,1,연예/문화
2,2,생활
3,3,경제
4,4,정치
...,...,...
3098,3098,생활
3099,3099,연예/문화
3100,3100,생활
3101,3101,경제
